In [37]:
import pandas as pd
import random
import numpy as np
from pprint import pprint

In [38]:
def frame_split(df, test_size_in_percent):
    
    test_size = round(test_size_in_percent/100 * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
        
    return train_df, test_df

In [39]:
df = pd.read_csv("extracsvAll.csv")

label_name = "type"
df = df.rename(columns={label_name:'label'})
df = df.round(1)
df.head()

,approxAnzahl,areaRatio,circleLike,contureDistance,conturenAnzahl,farPoint,keyPointsAnzahl,mean_val,middlepointDistance,minimal_ratio,minimal_ratio2,momentpointDistance,rectangleLike,seradity,seradity2,solidity,solidity2,steiner,label
0,26,0.0,0.4,1.0,3,65806,434,254.0,-2.8,2.7,1.0,-8.2,1.0,0.6,1.0,0.7,1.0,5.1,Dosenoeffner
1,27,0.1,0.4,1.1,2,29660,432,240.2,-1.0,2.0,1.7,24.3,1.2,0.7,1.0,0.5,1.0,30.2,Korkenzieher
2,20,0.1,0.5,1.3,2,7145,418,228.5,7.3,3.0,1.6,23.8,1.5,0.9,1.0,0.7,1.0,16.3,Flaschenoeffner
3,20,0.0,0.3,1.5,3,27976,322,249.4,7.5,1.6,1.8,-0.0,0.3,0.8,1.0,0.7,1.0,17.7,Dosenoeffner
4,19,0.0,0.4,1.1,2,56244,407,253.0,4.5,2.3,1.2,-7.6,1.0,0.7,1.0,0.6,1.0,19.2,Dosenoeffner


In [40]:
df_train, df_test = frame_split(df, test_size_in_percent=20)

test_labels = df_test["label"]
df_test = df_test.drop("label",axis=1)
df_test

,approxAnzahl,areaRatio,circleLike,contureDistance,conturenAnzahl,farPoint,keyPointsAnzahl,mean_val,middlepointDistance,minimal_ratio,minimal_ratio2,momentpointDistance,rectangleLike,seradity,seradity2,solidity,solidity2,steiner
346,20,0.1,0.3,0.8,4,8958,443,227.3,16.0,3.0,2.1,38.2,0.9,0.9,0.9,0.9,0.7,23.4
253,21,0.0,0.5,1.0,2,58613,432,250.9,-5.9,2.2,2.4,-15.2,1.2,0.6,1.0,0.5,0.9,15.1
580,18,0.3,0.3,1.2,3,9427,417,132.7,8.9,2.2,3.3,29.0,0.9,0.9,0.9,0.8,0.8,20.2
619,29,0.1,0.2,1.2,3,9316,424,213.1,23.7,2.2,1.4,55.9,0.3,0.8,0.9,0.9,1.0,14.8
76,12,0.3,0.5,0.8,2,8913,403,194.0,5.4,3.7,9.4,24.6,1.4,0.9,0.9,0.8,0.8,19.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,21,0.1,0.4,1.2,6,10536,444,204.8,7.2,2.6,5.1,31.0,1.1,0.8,1.0,0.7,1.0,13.3
310,26,0.1,0.4,1.1,7,43961,457,224.5,22.7,1.6,1.5,35.8,1.1,0.6,1.0,0.5,1.0,15.1
130,18,0.0,0.3,1.0,3,36362,419,251.5,15.9,3.3,1.9,54.2,0.9,0.7,1.0,0.8,1.0,5.0
135,17,0.0,0.4,1.4,3,5412,416,240.9,10.0,3.6,1.9,36.7,1.1,0.9,0.9,0.9,0.9,1.0


In [41]:
def wahrscheinlichkeiten(df, label_column):
    table = {}

    # determine values for the label
    value_counts = df[label_column].value_counts().sort_index()
    table["class_names"] = value_counts.index.to_numpy()
    table["class_counts"] = value_counts.values

    # determine probabilities for the features
    for feature in df.drop(label_column, axis=1).columns:
        table[feature] = {}

        # determine counts
        counts = df.groupby(label_column)[feature].value_counts()
        df_counts = counts.unstack(label_column)

        # add one count to avoid "problem of rare values"
        if df_counts.isna().any(axis=None):
            df_counts.fillna(value=0, inplace=True)
            df_counts += 1

        # calculate probabilities
        df_probabilities = df_counts / df_counts.sum()
        for value in df_probabilities.index:
            probabilities = df_probabilities.loc[value].to_numpy()
            table[feature][value] = probabilities
            
    return table

In [42]:
bayes_table = wahrscheinlichkeiten(df_train, label_column='label')
pprint(bayes_table)

{'approxAnzahl': {4: array([0.01282051, 0.0969163 , 0.03333333]),
                  6: array([0.0042735 , 0.01762115, 0.00416667]),
                  8: array([0.0042735 , 0.030837  , 0.00416667]),
                  9: array([0.0042735 , 0.01321586, 0.00416667]),
                  11: array([0.01709402, 0.03524229, 0.00416667]),
                  12: array([0.01709402, 0.07048458, 0.00416667]),
                  13: array([0.03418803, 0.02643172, 0.00833333]),
                  14: array([0.0042735 , 0.05286344, 0.00416667]),
                  15: array([0.03418803, 0.04405286, 0.01666667]),
                  16: array([0.02991453, 0.06167401, 0.01666667]),
                  17: array([0.05128205, 0.03964758, 0.00833333]),
                  18: array([0.1025641 , 0.07929515, 0.02083333]),
                  19: array([0.06410256, 0.04405286, 0.025     ]),
                  20: array([0.1025641 , 0.04405286, 0.05416667]),
                  21: array([0.09401709, 0.03524229, 0.05416667]),

## 2. Step of the Algorithm

In [43]:
def example(row,bayes_table):
    
        class_estimates = bayes_table["class_counts"]
        for feature in row.index:

            try:
                value = row[feature]
                probabilities = bayes_table[feature][value]
                class_estimates = class_estimates * probabilities

            # skip in case "value" only occurs in test set but not in train set
            # (i.e. "value" is not in "lookup_table")
            except KeyError:
                continue

        index_max_class = class_estimates.argmax()
        prediction = bayes_table["class_names"][index_max_class]
        
        return prediction

In [44]:
def calculate_keynumbers_bayes(df, bayes_table):

        df["classification"] = df.apply(example, axis=1, args=(bayes_table,))
        df["classification_correct"] = df["classification"] == df["label"]
        overall_accuracy = df["classification_correct"].mean()

        label_column = df['label']
        unique_classes, _ = np.unique(label_column, return_counts=True)
            
        keynumbers = []

        for uc in unique_classes:
                
            tp_df = df.loc[(df['label'] == uc) & (df["classification_correct"] == True)]
            fp_df = df.loc[(df['label'] == uc) & (df["classification_correct"] == False)]
            tn_df = df.loc[(df['label'] != uc) & (df["classification_correct"] == True)]
            fn_df = df.loc[(df['label'] != uc) & (df["classification_correct"] == False)]

            true_positiv, _ = tp_df.shape
            false_positiv, _ = fp_df.shape
            true_negative, _ = tn_df.shape
            false_negative, _ = fn_df.shape

            recall = true_positiv / (true_positiv+false_negative)
            precision = true_positiv / (true_positiv+false_positiv)
            f1score = 2 * (precision * recall) / (precision + recall)

            keynumbers.append({"Label":uc, "recall":recall, "precision":precision, "f1score":f1score})
            
            
        return overall_accuracy, keynumbers

# Check Accuracy

In [45]:
accuracy, keynumbers = calculate_keynumbers_bayes(df_test, bayes_table)
accuracy

KeyError: 'label'

In [ ]:
keynumbers = pd.DataFrame(keynumbers)
keynumbers